<img src="https://1000logos.net/wp-content/uploads/2017/11/Zillow-Logo.png" title="Zillow Logo"/>

***

- imports that we will be using for this data set

In [1]:
# fetches the data
import acquire
# credentials file to access the data
import env
# Imports functions necessary to run visuals and hides unnecessary code
import wrangle

# coding 
import math
import numpy as np
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data
import scipy.stats
import scipy
import os

# needed for modeling
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer
from sklearn.metrics import explained_variance_score
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cluster import KMeans

***

Fips dictionary:
- 6037.0 = Los Angeles,CA
- 6059.0 = Orange,CA
- 6111.0 = Ventura,CA

# <span style="color:blue">Plan</span>
- Acquire Data
- Clean and Prep the Data
- Explore the features of the Zillow data
- Evaluate Data
- Test Data

# <span style="color:blue">Acquiring Data</span>

In [2]:
# importing and aquiring data set
df = acquire.get_zillow_data()

### - Summary

In [3]:
# summary function for DataFrame
wrangle.summarize(df)

--- Shape: (71858, 69)
--- Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71858 entries, 0 to 71857
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   propertylandusetypeid         71858 non-null  float64
 1   parcelid                      71858 non-null  int64  
 2   storytypeid                   47 non-null     float64
 3   typeconstructiontypeid        223 non-null    float64
 4   heatingorsystemtypeid         46680 non-null  float64
 5   buildingclasstypeid           0 non-null      float64
 6   architecturalstyletypeid      207 non-null    float64
 7   airconditioningtypeid         23069 non-null  float64
 8   id                            71858 non-null  int64  
 9   basementsqft                  47 non-null     float64
 10  bathroomcnt                   71858 non-null  float64
 11  bedroomcnt                    71858 non-null  float64
 12  buildingqualitytypeid      

***

# <span style="color:blue">Preparing & Cleaning Data</span>
- all these functions will be found in the explore.py file

###  What percentage of data is missing per column?

In [4]:
#looking at percentage of null values by column
wrangle.nulls_by_columns(df).sort_values(by= 'percent', ascending=False)

,count,percent
buildingclasstypeid,71858,1.000000
buildingclassdesc,71858,1.000000
finishedsquarefeet13,71856,0.999972
finishedsquarefeet15,71847,0.999847
storydesc,71811,0.999346
basementsqft,71811,0.999346
storytypeid,71811,0.999346
yardbuildingsqft26,71788,0.999026
finishedsquarefeet6,71692,0.997690
fireplaceflag,71686,0.997606


### Takeaways:

- we can see that there is a large percentage of information missing in alot of the columns (ranging from column regionidneighborhood with %60 data missing to buildingclasstypeid with %100 of the data missing.

### Functions to clean data:

- lets build a function to address the columns with large percentage of missing data.

In [9]:
k = 1.5
cols = ['bathroomcnt', 'bedroomcnt','calculatedfinishedsquarefeet','yearbuilt','lotsizesquarefeet','lotsizesquarefeet']    

def handle_outliers(df, cols, k):
    """this will eliminate most outliers, use a 1.5 k value if unsure because it is the most common, make sure to define cols value as the features
    you want the outliers to be handled. this should be done before running the function and outiside of it"""

    
    # Create placeholder dictionary for each columns bounds
    bounds_dict = {}
   
    for col in cols:
        # get necessary iqr values
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        upper_bound =  q3 + k * iqr
        lower_bound =  q1 - k * iqr

        #store values in a dictionary referencable by the column name
        #and specific bound
        bounds_dict[col] = {}
        bounds_dict[col]['upper_bound'] = upper_bound
        bounds_dict[col]['lower_bound'] = lower_bound

    for col in cols:
        #retrieve bounds
        col_upper_bound = bounds_dict[col]['upper_bound']
        col_lower_bound = bounds_dict[col]['lower_bound']

        #remove rows with an outlier in that column
        df = df[(df[col] < col_upper_bound) & (df[col] > col_lower_bound)]
        
    return df
df = handle_outliers(df, cols, k)
df

,propertylandusetypeid,parcelid,storytypeid,typeconstructiontypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,typeconstructiondesc,storydesc,propertylandusedesc
0,261.0,14297519,NaN,NaN,NaN,NaN,NaN,NaN,1727539,NaN,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
1,261.0,17052889,NaN,NaN,NaN,NaN,NaN,NaN,1387261,NaN,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
2,261.0,14186244,NaN,NaN,NaN,NaN,NaN,NaN,11677,NaN,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
3,261.0,12177905,NaN,NaN,2.0,NaN,NaN,NaN,2288172,NaN,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,NaN,NaN,Single Family Residential
5,266.0,17143294,NaN,NaN,NaN,NaN,NaN,NaN,1447245,NaN,...,5,-0.020526,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71851,261.0,12412492,NaN,NaN,2.0,NaN,NaN,NaN,2274245,NaN,...,77607,0.001082,2017-09-19,NaN,NaN,NaN,Central,NaN,NaN,Single Family Residential
71854,261.0,17239384,NaN,NaN,NaN,NaN,NaN,NaN,2968375,NaN,...,77610,0.013209,2017-09-21,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
71855,261.0,12773139,NaN,NaN,2.0,NaN,NaN,1.0,1843709,NaN,...,77611,0.037129,2017-09-21,Central,NaN,NaN,Central,NaN,NaN,Single Family Residential
71856,261.0,12826780,NaN,NaN,2.0,NaN,NaN,NaN,1187175,NaN,...,77612,0.007204,2017-09-25,NaN,NaN,NaN,Central,NaN,NaN,Single Family Residential


## Takeaways
- Before dropping nulls, we had (71858 rows, 69 cols).
- After dropping nulls, we ended with (49964 rows × 69 columns).

### Lets see what were left with

In [10]:
# a view a how many null values exist in each column
wrangle.nulls_by_columns(df).sort_values(by= 'percent', ascending=False)

,count,percent
buildingclassdesc,49964,1.000000
buildingclasstypeid,49964,1.000000
finishedsquarefeet15,49964,1.000000
finishedsquarefeet13,49963,0.999980
basementsqft,49938,0.999480
storydesc,49938,0.999480
storytypeid,49938,0.999480
yardbuildingsqft26,49921,0.999139
architecturalstyletypeid,49896,0.998639
architecturalstyledesc,49896,0.998639


In [11]:
df.columns

Index(['propertylandusetypeid', 'parcelid', 'storytypeid',
       'typeconstructiontypeid', 'heatingorsystemtypeid',
       'buildingclasstypeid', 'architecturalstyletypeid',
       'airconditioningtypeid', 'id', 'basementsqft', 'bathroomcnt',
       'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr',
       'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
       'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr',
       'unitcnt', 'yardbuildings

### Lets adress the rest of the  null values

### columns to remove:(column, reason for removal)
removing 
- (id, id.1, parcelid,'propertylandusetypeid','buildingqualitytypeid'), id is not necessary for our algorithms and will confuse any models from here on forward.
- (fullbathcnt,calculatedbathnbr,roomcnt),  any room room count other the bedroomcnt or bathroomcnt is not necessary considering that they return similar information if not combined info.
- (propertyzoningdesc,rawcensustractandblock,regionidcounty,censustractandblock), considering that fips is being kept for region identification purposes, these columns are not necessary.
- (assessmentyear, landtaxvaluedollarcnt, taxamount, transactiondate), considering that we have already filtered out the data to only return back information for the year 2017, and are keeping taxvaluedollarcnt, these columns are not necessary because this information can be obtained through the data that we will be keeping.
- (heatingorsystemdesc,finishedsquarefeet12,propertylandusedesc,'propertycountylandusecode','unitcnt'), calculatedfinishedsquarefeet already covers this info and heatingorsystemid already identifies this information numerically.

In [27]:
def drop_columns(df):
    df = df.drop(columns=['heatingorsystemtypeid','buildingqualitytypeid','id','parcelid','calculatedbathnbr','propertylandusetypeid','fullbathcnt','propertyzoningdesc','rawcensustractandblock','regionidcounty',
    'roomcnt','structuretaxvaluedollarcnt','assessmentyear','landtaxvaluedollarcnt','taxamount','censustractandblock',
    'id.1','transactiondate','heatingorsystemdesc','finishedsquarefeet12','propertylandusedesc','propertycountylandusecode','unitcnt'])
    return df


def split(df):
    train_and_validate, test = train_test_split(df, random_state=13, test_size=.15)
    train, validate = train_test_split(train_and_validate, random_state=13, test_size=.2)

    print('Train: %d rows, %d cols' % train.shape)
    print('Validate: %d rows, %d cols' % validate.shape)
    print('Test: %d rows, %d cols' % test.shape)
    
    return train, validate, test    

def handle_missing_values(df, prop_required_column, prop_required_row):
    #this piece of code allows us to handle the missing data and get rid of it, both in the columns and in the rows(so that we can analize better).
    print ('Before dropping nulls, %d rows, %d cols' % df.shape)
    n_required_column = round(df.shape[0] * prop_required_column)
    n_required_row = round(df.shape[1] * prop_required_row)
    df = df.dropna(axis=0, thresh=n_required_row)
    df = df.dropna(axis=1, thresh=n_required_column)
    df = drop_columns(df)
    print('After dropping nulls. %d rows. %d cols' % df.shape)
    return df

def get_exploration_data(df):
    #drops rows and columns with more than %50 data missing
    print('Before dropping nulls, %d rows, %d cols' % df.shape)
    df = handle_missing_values(df, prop_required_column=.5, prop_required_row=.5)
    print('After dropping nulls, %d rows, %d cols' % df.shape)
    
    train, validate, test = split(df)
    
    return train, validate, test
get_exploration_data(df)

Before dropping nulls, 49964 rows, 69 cols
Before dropping nulls, 49964 rows, 69 cols
After dropping nulls. 44679 rows. 12 cols
After dropping nulls, 44679 rows, 12 cols
Train: 30381 rows, 12 cols
Validate: 7596 rows, 12 cols
Test: 6702 rows, 12 cols


(       bathroomcnt  bedroomcnt  calculatedfinishedsquarefeet    fips  \
 57528          1.0         2.0                         950.0  6037.0   
 2250           3.5         5.0                        3128.0  6111.0   
 20338          2.5         3.0                        1536.0  6059.0   
 31373          2.5         2.0                        1194.0  6059.0   
 59529          2.0         4.0                        2288.0  6059.0   
 ...            ...         ...                           ...     ...   
 16134          3.0         3.0                        1581.0  6037.0   
 53368          2.0         2.0                        2258.0  6037.0   
 50836          2.0         3.0                        1992.0  6111.0   
 28844          3.0         3.0                        1915.0  6037.0   
 47863          3.0         4.0                        2570.0  6037.0   
 
          latitude    longitude  lotsizesquarefeet  regionidcity  regionidzip  \
 57528  34687977.0 -118138159.0          

# <span style="color:blue">Split Data</span>

In [30]:
# get train to expolore 
train, validate, test = get_exploration_data(df)
# seeing what the train split dataset
train.info()

Before dropping nulls, 49964 rows, 69 cols
Before dropping nulls, 49964 rows, 69 cols
After dropping nulls. 44679 rows. 12 cols
After dropping nulls, 44679 rows, 12 cols
Train: 30381 rows, 12 cols
Validate: 7596 rows, 12 cols
Test: 6702 rows, 12 cols
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30381 entries, 57528 to 47863
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   30381 non-null  float64
 1   bedroomcnt                    30381 non-null  float64
 2   calculatedfinishedsquarefeet  30381 non-null  float64
 3   fips                          30381 non-null  float64
 4   latitude                      30381 non-null  float64
 5   longitude                     30381 non-null  float64
 6   lotsizesquarefeet             30381 non-null  float64
 7   regionidcity                  29991 non-null  float64
 8   regionidzip                   30380 non-n

In [31]:
wrangle.nulls_by_columns(train).sort_values(by= 'percent', ascending=False)

,count,percent
regionidcity,390,0.012837
regionidzip,1,0.000033
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000
calculatedfinishedsquarefeet,0,0.000000
fips,0,0.000000
latitude,0,0.000000
longitude,0,0.000000
lotsizesquarefeet,0,0.000000
yearbuilt,0,0.000000


***

In [32]:
validate

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,regionidcity,regionidzip,yearbuilt,taxvaluedollarcnt,logerror
50990,4.0,4.0,2909.0,6037.0,34154272.0,-118072688.0,11592.0,47019.0,96295.0,1950.0,789122.0,-0.003082
71369,2.0,3.0,2201.0,6037.0,34068700.0,-118341707.0,7161.0,12447.0,96017.0,1924.0,1305806.0,0.014879
40712,2.0,3.0,1613.0,6111.0,34205775.0,-118874393.0,9500.0,34278.0,96383.0,1966.0,351004.0,0.000692
13313,3.0,5.0,2560.0,6037.0,34071706.0,-118312205.0,9539.0,12447.0,95985.0,1920.0,107455.0,-0.091738
15417,2.0,2.0,1517.0,6037.0,34188343.0,-118451323.0,7500.0,12447.0,96420.0,1939.0,377969.0,-0.017084
...,...,...,...,...,...,...,...,...,...,...,...,...
28605,3.0,4.0,1779.0,6037.0,34563318.0,-118097594.0,6295.0,40227.0,97328.0,1987.0,136362.0,-0.002565
54387,2.0,3.0,1493.0,6037.0,33899729.0,-118227917.0,4948.0,24174.0,96090.0,1951.0,46247.0,0.026311
14990,2.0,2.0,1102.0,6037.0,34688998.0,-118083130.0,6227.0,5534.0,97318.0,1987.0,140000.0,-0.394014
34851,3.0,4.0,2273.0,6037.0,34267869.0,-118585031.0,9005.0,12447.0,96346.0,1972.0,320592.0,0.012950


In [33]:
test

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,regionidcity,regionidzip,yearbuilt,taxvaluedollarcnt,logerror
37437,1.0,2.0,697.0,6111.0,34274402.0,-119273072.0,2296.0,34543.0,97081.0,1928.0,327744.0,0.031970
51668,1.0,2.0,924.0,6037.0,33823346.0,-118114326.0,5629.0,46298.0,96242.0,1950.0,457702.0,-0.035976
33794,1.0,3.0,1026.0,6037.0,33883231.0,-118253054.0,5098.0,24174.0,96090.0,1950.0,319803.0,-0.043604
5537,2.0,3.0,1277.0,6059.0,33766617.0,-117900387.0,7000.0,47568.0,97006.0,1961.0,441633.0,0.491767
65612,2.0,2.0,1703.0,6037.0,34159186.0,-118417863.0,6056.0,12447.0,96424.0,1952.0,114341.0,-0.127774
...,...,...,...,...,...,...,...,...,...,...,...,...
676,2.0,3.0,2898.0,6037.0,34168443.0,-118064600.0,11770.0,47695.0,96278.0,1943.0,330723.0,0.028165
28428,3.0,3.0,2601.0,6059.0,33582006.0,-117641952.0,7000.0,12773.0,96996.0,1994.0,497382.0,-0.057161
57507,2.0,3.0,1358.0,6037.0,34201617.0,-118621599.0,8447.0,12447.0,96342.0,1956.0,100462.0,0.194552
29286,1.0,3.0,983.0,6037.0,34575829.0,-118094140.0,6089.0,40227.0,97328.0,1954.0,43506.0,-0.116010


***

In [ ]:
# Declaring the points for first line plot
X1 = train.bedroomcnt 
Y1 = train.fips
# plotting the first plot
plt.plot(X1, Y1, label = "plot 1") 
# Declaring the points for second line plot
X2 = train.bathroomcnt 
Y2 = train.fips
# plotting the second plot 
plt.plot(X2, Y2, label = "plot 2") 
  
# Labeling the X-axis 
plt.xlabel('X-axis') 
# Labeling the Y-axis 
plt.ylabel('Y-axis') 
# Give a title to the graph
plt.title('Two plots on the same graph') 
  
# Show a legend on the plot 
plt.legend() 
 
plt.show()

In [ ]:
train.head()
